In [1]:
# Some simple first steps
# - Aim is that notebook analysis can start with this simple code

from Malikowski.Adoption import Adoption

#-- specify the courses you want to analyse
#   Option to include (maybe)
#   - course = []  pass a list of courses
#   - PERIOD = []  pass a list of terms/periods e.g. [ 2015_1 2014_3 ]
#   - nothing - get all of them
# - ALTERNATIVE
#   Just support list of courses.  Use other course specific means to identify the specific
#   list of courses you're interested in for any analysis.

#courses = [6082]
#courses = []
courses = [ 6082, 4688, 4381]

#-- get the data
#   Eventually there will be different classess
#   - Adoption - standard adoption of features - from course_module table
#   - Usage - based on clicks of students
#   - ExtendedUsage - the extension to Malikowski where we put in extras
#          e.g. staff/student communication/content etc
# ...
model = Adoption()

model.getCourses( courses )

#-- Generates a range of data members
#   - df - data frame rows = count for 1 malikowski category for a course.
#   - malikowskiGroup - possibly interim stage
#   - malikowski - data frame with all course malikowski usage in one row
model.df

In [2]:
model.malikowskiGroup

In [3]:
model.malikowski
# (possible) Changes to do at this stage
# - make sure that cbi and evaluation columns get added, even if 0
# - maybe add percentage here (what about remove percentage above?)
# - index this on course, not malikowski

In [4]:
#**************************************
# Everything from here is exploration and updates
# - some early visualisation experiments below


group = model.df.groupby(['course','shortname','fullname','malikowski']).sum().reset_index()
group = model.malikowskiGroup
group

In [5]:
#*****
# Add (0x,0%) rows to malikowski (via malikowskiGroup) for missing categories

#-- get list of course / malikowski that is missing
#-- open empty array
missing=[]

group = model.malikowskiGroup;
#- get list of uniqe course ids
courses = group.course.unique()

#-- for each course in the data frame
for course in courses:
#    print ( "course is " + str(course))
    # get data frame with the present malikowski categories for the current course
    present=group.loc[group['course'] == course, ['malikowski','shortname','fullname']]
    # grab the short/fullname
    shortName = present.iloc[0].shortname
    fullName = present.iloc[0].fullname
    print(present)
    # get a list of the malikowski categories for this course
    presentList = present['malikowski'].tolist()
    
#    print(model.allCategories)
#    print(presentList)
    # get a list of categories not present for this course
    notThere = list(set(model.allCategories) - set(presentList))
#    print(notThere) 

    for cat in notThere:
        # add rows to missing
        row = { 'course':course,  'malikowski':cat, 'shortname':shortName, 'fullname':fullName,
                'x':0, 'percent':0}
        missing.append( row)
            
import pandas as pd
#missingDF = DataFrame(missing)
missing
missingDF = pd.DataFrame(missing)
missingDF

In [6]:
#model.malikowskiGroup = model.malikowskiGroup.append( missingDF)

total = pd.concat( [model.malikowskiGroup,missingDF], axis=0 )
total.reset_index(drop=True,inplace=True)
total

In [7]:
#model.malikowskiCourseDF

model.malikowskiGroup

In [8]:
#next = model.malikowskiGroup.pivot_table('%',['course','shortname','fullname'],'malikowski')
#next
#next = model.malikowskiGroup.pivot_table('x',['course','shortname','fullname'],'malikowski')
#next.reset_index(drop=False,inplace=True)
#next
model.malikowski

In [9]:
import matplotlib.pyplot as plt

In [10]:
import matplotlib
matplotlib.style.use('ggplot')
%matplotlib inline

In [11]:
final = next.drop( ['course','fullname'], axis=1)

In [ ]:

final.plot.bar(stacked=True)

In [ ]:
# adapting some stuff from http://www.randalolson.com/2014/06/28/how-to-make-beautiful-data-visualizations-in-python-with-matplotlib/
f, ax1 = plt.subplots(1,figsize=(12,9))

bar_width = 0.75

bar_l = [i+1 for i in range(len(final['assessment']))]

tick_pos = [i+(bar_width/2) for i in bar_l]

ax1.bar( bar_l, final['content'], label='content',
        width=bar_width,alpha=0.5,color='#F4561D')
ax1.bar( bar_l, final['communication'], label='communication',bottom=final['content'],
        width=bar_width,alpha=0.5,color='#F1911E')
ax1.bar( bar_l, final['assessment'], label='assessment',
        bottom=[i+j for i,j in zip(final['content'],final['communication'])],
        width=bar_width,alpha=0.5,color='#F1BD1A')
#ax1.bar( bar_l, final['evaluation'], label='evaluation',
 #       bottom=[i+j for i,j in zip(final['content'],final['communication'],final['assessment'])],
  #      width=bar_width,alpha=0.5,color='#FF0000')

plt.xticks( tick_pos, final['shortname'])
ax1.set_ylabel( "Category adoption")
ax1.set_xlabel( "EDC3100 offering")
plt.legend(loc='upper left')

#plt.plot()
plt.xlim([min(tick_pos)-bar_width, max(tick_pos)+bar_width])